## 抓取新浪微博新闻列表

In [1]:
import requests
from bs4 import BeautifulSoup
import re

res = requests.get('http://news.sina.com.cn/china/')
res.encoding = 'utf-8'
#res.text

In [4]:
soup = BeautifulSoup(res.text, 'html.parser')

newsary = []
for link in soup.select('.news-item'):
    if len(link.select('h2 a')) > 0:
        newsary.append(getArticle(link.select('h2 a')[0]['href']))

## 建立文章抓取函数

In [3]:
def getArticle(url):
    res = requests.get(url)
    res.encoding = 'utf-8'
    
    soup = BeautifulSoup(res.text, 'html.parser')
    dic = {}
    
    dic['title'] = soup.select('#artibodyTitle')[0].text
    dic['content'] = ' '.join(soup.select('#artibody')[0].text.split())
    dic['source'] = soup.select('#navtimeSource')[0].text
    dic['keyword'] = soup.select('.article-keywords')[0].text
    return dic

#getArticle('http://news.sina.com.cn/c/sd/2016-11-29/doc-ifxyawxa3005371.shtml')

## 建立DataFrame

In [5]:
import pandas
df = pandas.DataFrame(newsary)
df.head(10)

,content,keyword,source,title
0,原标题：南京机场88个航班延误，启动橙色预警 现代快报讯（记者 刘伟娟）受南京、广州、福州等...,\n文章关键词：\n南京机场 橙色预警 黄色预警\n,2017年06月10日21:10\t\t\n现代快报\n,南京机场88个航班延误 黄色预警升级为橙色预警
1,原标题：南京单日降雨量刷历史新高 5000多人应急抢险 南京市防汛进入紧急状态，出动5000...,\n文章关键词：\n抢险 降雨量\n,2017年06月10日20:05\t\t\n央视新闻\n,南京单日降雨量刷新历史记录 5千多人应急抢险
2,原标题：长江流域梅雨期提前到来 三峡等水库加快腾库备汛 中新社武汉6月10日电 （记者 徐金...,\n文章关键词：\n水库 长江\n,2017年06月10日19:24\t\t\n中国新闻网\n,长江流域梅雨期提前到来 三峡水库加快腾库备汛
3,原标题：江苏8市大暴雨南京破纪录 今夜雨水渐止 中国天气网讯 今天（10日）江苏迎来了大范围...,\n文章关键词：\n大暴雨 雨水\n,2017年06月10日19:14\t\t\n中国天气网\n,江苏8市大暴雨南京破纪录 今夜雨水将减弱停止
4,原标题：大连交警整治夜间飙车 扣留法拉利、玛莎拉蒂等14台豪车 大连交警整治夜间“飙车”行为...,\n文章关键词：\n豪车 交警\n,2017年06月10日19:03\t\t\n中国新闻网\n,大连整治夜间飙车 扣法拉利玛莎拉蒂等14台豪车
5,原标题：桂林尧山缆车停电导致游客被困 图为桂林尧山景区。 赵琳露 摄 中新网桂林6月10日电...,\n文章关键词：\n缆车 停电 游客\n,2017年06月10日17:22\t\t\n中国新闻网\n,广西桂林缆车停电致约200名游客被困
6,原标题：北京首条磁浮线路开始联调联试 资料图 央视网消息：从本周开始，北京市域铁路S1线开始...,\n文章关键词：\n磁浮线路 运营\n,2017年06月10日16:15\t\t\n央视新闻\n,北京首条磁浮线路开始联调联试 有望年内试运营
7,原标题：长三角地区遭遇特大雷暴雨突袭 江苏拉响三级灾害响应 大雨当前，南京当地城管奋战在抢险...,\n文章关键词：\n雷暴雨 长三角\n,2017年06月10日15:07\t\t\n中国新闻网\n,长三角遭特大雷暴雨突袭 江苏拉响三级灾害响应
8,原标题：台湾花莲直升机坠机 导演齐柏林等3人遇难 海外网6月10日援引台湾“中央社”消息称，...,\n文章关键词：\n直升机 台湾 坠机\n,2017年06月10日15:10\t\t\n新华报业网\n,金马导演齐柏林拍摄时坠机遇难 两天前开发布会
9,中新社西宁6月10日电 （记者 罗云鹏）青藏高原野生动物园10日对外表示，位于该园的中国唯一...,\n文章关键词：\n雪豹 青藏高原 幼崽\n,2017年06月10日14:43\t\t\n中国新闻网\n,中国人工繁育成活首只雪豹满周岁 征名活动启动


## 整理文章关键词

In [10]:
df['keyword'].head(3)

0    \n文章关键词：\n南京机场 橙色预警 黄色预警\n
1            \n文章关键词：\n抢险 降雨量\n
2             \n文章关键词：\n水库 长江\n
Name: keyword, dtype: object

In [178]:
#df['keyword'].map(lambda e: e.split('：')[1].split())#根据冒号切割取后面部分

In [11]:
df['keyword'] = df['keyword'].map(lambda e: e.split('：')[1].split())#记得修改后返回给原来数据

In [12]:
df.head(3)

,content,keyword,source,title,datetime,from
0,原标题：南京机场88个航班延误，启动橙色预警 现代快报讯（记者 刘伟娟）受南京、广州、福州等...,"[南京机场, 橙色预警, 黄色预警]",2017年06月10日21:10\t\t\n现代快报\n,南京机场88个航班延误 黄色预警升级为橙色预警,2017年06月10日21:10,现代快报
1,原标题：南京单日降雨量刷历史新高 5000多人应急抢险 南京市防汛进入紧急状态，出动5000...,"[抢险, 降雨量]",2017年06月10日20:05\t\t\n央视新闻\n,南京单日降雨量刷新历史记录 5千多人应急抢险,2017年06月10日20:05,央视新闻
2,原标题：长江流域梅雨期提前到来 三峡等水库加快腾库备汛 中新社武汉6月10日电 （记者 徐金...,"[水库, 长江]",2017年06月10日19:24\t\t\n中国新闻网\n,长江流域梅雨期提前到来 三峡水库加快腾库备汛,2017年06月10日19:24,中国新闻网


## 使用正规表达法抽取时间与源数据

In [13]:
df['source'].head()

0     2017年06月10日21:10\t\t\n现代快报\n
1     2017年06月10日20:05\t\t\n央视新闻\n
2    2017年06月10日19:24\t\t\n中国新闻网\n
3    2017年06月10日19:14\t\t\n中国天气网\n
4    2017年06月10日19:03\t\t\n中国新闻网\n
Name: source, dtype: object

In [14]:
df['source'].map(lambda e: e.split()).head()

0     [2017年06月10日21:10, 现代快报]
1     [2017年06月10日20:05, 央视新闻]
2    [2017年06月10日19:24, 中国新闻网]
3    [2017年06月10日19:14, 中国天气网]
4    [2017年06月10日19:03, 中国新闻网]
Name: source, dtype: object

In [185]:
#df['source'].str.extract('(\d+年\d+月\d+日\d+:\d+)\t\t\n(\w+)\n')#正规表达式

In [15]:
df[['datetime', 'from']] = df['source'].str.extract('(\d+年\d+月\d+日\d+:\d+)\t\t\n(\w+)\n', expand =False)#分别放到两个栏位中

In [16]:
df.head(3)

,content,keyword,source,title,datetime,from
0,原标题：南京机场88个航班延误，启动橙色预警 现代快报讯（记者 刘伟娟）受南京、广州、福州等...,"[南京机场, 橙色预警, 黄色预警]",2017年06月10日21:10\t\t\n现代快报\n,南京机场88个航班延误 黄色预警升级为橙色预警,2017年06月10日21:10,现代快报
1,原标题：南京单日降雨量刷历史新高 5000多人应急抢险 南京市防汛进入紧急状态，出动5000...,"[抢险, 降雨量]",2017年06月10日20:05\t\t\n央视新闻\n,南京单日降雨量刷新历史记录 5千多人应急抢险,2017年06月10日20:05,央视新闻
2,原标题：长江流域梅雨期提前到来 三峡等水库加快腾库备汛 中新社武汉6月10日电 （记者 徐金...,"[水库, 长江]",2017年06月10日19:24\t\t\n中国新闻网\n,长江流域梅雨期提前到来 三峡水库加快腾库备汛,2017年06月10日19:24,中国新闻网


In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96 entries, 0 to 95
Data columns (total 6 columns):
content     96 non-null object
keyword     96 non-null object
source      96 non-null object
title       96 non-null object
datetime    95 non-null object
from        95 non-null object
dtypes: object(6)
memory usage: 4.6+ KB


## 转换时间格式

In [18]:
df['datetime'] = pandas.to_datetime(df['datetime'], format = '%Y年%m月%d日%H:%M')

In [19]:
df['datetime'].head()

0   2017-06-10 21:10:00
1   2017-06-10 20:05:00
2   2017-06-10 19:24:00
3   2017-06-10 19:14:00
4   2017-06-10 19:03:00
Name: datetime, dtype: datetime64[ns]

In [24]:
#df['datetime'].map(lambda e: e.year)
#df['datetime'].map(lambda e: e.month)  #转换为时间格式之后可以直接提取年月日

In [25]:
df.head(3)

,content,keyword,source,title,datetime,from
0,原标题：南京机场88个航班延误，启动橙色预警 现代快报讯（记者 刘伟娟）受南京、广州、福州等...,"[南京机场, 橙色预警, 黄色预警]",2017年06月10日21:10\t\t\n现代快报\n,南京机场88个航班延误 黄色预警升级为橙色预警,2017-06-10 21:10:00,现代快报
1,原标题：南京单日降雨量刷历史新高 5000多人应急抢险 南京市防汛进入紧急状态，出动5000...,"[抢险, 降雨量]",2017年06月10日20:05\t\t\n央视新闻\n,南京单日降雨量刷新历史记录 5千多人应急抢险,2017-06-10 20:05:00,央视新闻
2,原标题：长江流域梅雨期提前到来 三峡等水库加快腾库备汛 中新社武汉6月10日电 （记者 徐金...,"[水库, 长江]",2017年06月10日19:24\t\t\n中国新闻网\n,长江流域梅雨期提前到来 三峡水库加快腾库备汛,2017-06-10 19:24:00,中国新闻网


## 刪除Source

In [26]:
del df['source']

In [27]:
df.head(3)

,content,keyword,title,datetime,from
0,原标题：南京机场88个航班延误，启动橙色预警 现代快报讯（记者 刘伟娟）受南京、广州、福州等...,"[南京机场, 橙色预警, 黄色预警]",南京机场88个航班延误 黄色预警升级为橙色预警,2017-06-10 21:10:00,现代快报
1,原标题：南京单日降雨量刷历史新高 5000多人应急抢险 南京市防汛进入紧急状态，出动5000...,"[抢险, 降雨量]",南京单日降雨量刷新历史记录 5千多人应急抢险,2017-06-10 20:05:00,央视新闻
2,原标题：长江流域梅雨期提前到来 三峡等水库加快腾库备汛 中新社武汉6月10日电 （记者 徐金...,"[水库, 长江]",长江流域梅雨期提前到来 三峡水库加快腾库备汛,2017-06-10 19:24:00,中国新闻网


In [29]:
df = df[['from','title', 'content', 'keyword', 'datetime']]#调整列顺序
df.head()

,from,title,content,keyword,datetime
0,现代快报,南京机场88个航班延误 黄色预警升级为橙色预警,原标题：南京机场88个航班延误，启动橙色预警 现代快报讯（记者 刘伟娟）受南京、广州、福州等...,"[南京机场, 橙色预警, 黄色预警]",2017-06-10 21:10:00
1,央视新闻,南京单日降雨量刷新历史记录 5千多人应急抢险,原标题：南京单日降雨量刷历史新高 5000多人应急抢险 南京市防汛进入紧急状态，出动5000...,"[抢险, 降雨量]",2017-06-10 20:05:00
2,中国新闻网,长江流域梅雨期提前到来 三峡水库加快腾库备汛,原标题：长江流域梅雨期提前到来 三峡等水库加快腾库备汛 中新社武汉6月10日电 （记者 徐金...,"[水库, 长江]",2017-06-10 19:24:00
3,中国天气网,江苏8市大暴雨南京破纪录 今夜雨水将减弱停止,原标题：江苏8市大暴雨南京破纪录 今夜雨水渐止 中国天气网讯 今天（10日）江苏迎来了大范围...,"[大暴雨, 雨水]",2017-06-10 19:14:00
4,中国新闻网,大连整治夜间飙车 扣法拉利玛莎拉蒂等14台豪车,原标题：大连交警整治夜间飙车 扣留法拉利、玛莎拉蒂等14台豪车 大连交警整治夜间“飙车”行为...,"[豪车, 交警]",2017-06-10 19:03:00


## 将整理好的数据储存回Excel

In [30]:
df.to_excel('news.xlsx')